## Загрузка библиотек, данных

In [ ]:
!pip install -U -q PyDrive
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=c2816011b18677f0da5e890d7adee1fde658ab124791d5ea317c8f8a3b750b03
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
import numpy as np
import pandas as pd

from google.colab import drive
from google.colab import files
import os

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sequence, to_date, explode, col
spark = SparkSession \
.builder \
.appName("Python Spark SQL basic example") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()

In [ ]:
# просмотр, где находится каталог с файлами на COLAB
drive.mount('/content/drive')

Mounted at /content/drive


## Предобработка данных

In [ ]:
data_posting_table = pd.read_excel('/content/drive/My Drive/Colab Notebooks/posting_table.xlsx')
data_clients_info = pd.read_excel('/content/drive/My Drive/Colab Notebooks/clients_info.xlsx')
data_pet_chk_20_pet_chk_21 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/pet_chk_20_pet_chk_21.xlsx')

In [ ]:
data_posting_table

,BUSINESS_DT,ACCOUNT_DEBIT_ID,POSTING_AMT,ACCOUNT_CREDIT_ID
0,2020-01-01,1,1000,2
1,2020-01-05,2,500,1
2,2020-02-04,2,500,1
3,2020-03-04,1,1000,2


In [ ]:
data_clients_info.head(5)

,row_id,registration_date,fst_name,last_name,mid_name,sex_mf,card_no
0,1,2019-04-01,Иванов,Петр,Сергеевич,male,2675354
1,2,2019-05-02,Петров,Сергей,Иванович,male,2675355
2,3,2018-06-03,Семенов,Семен,Семенович,male,2675356
3,4,2017-07-05,Сергеев,Иван,Петрович,male,2675357
4,5,2016-08-06,Синицина,Ольга,Ивановна,female,2675358


In [ ]:
data_pet_chk_20_pet_chk_21.head(5)

,card_no,operation_date,cheque_id,vol_ai92,vol_ai95,vol_ai98,vol_gaz,vol_diesel,vol_other,wt_ai92,...,a_ai98,a_gaz,a_diesel,a_other,ta_ai92,ta_ai95,ta_ai98,ta_gaz,ta_diesel,ta_other
0,2675354,2020-01-01,1,NaN,NaN,NaN,NaN,NaN,10.0,0.0,...,0.0,0,0,377.0,0.0,0.000,0.000,0.0,0,369.46
1,2675355,2020-01-02,2,NaN,NaN,NaN,40.0,NaN,NaN,0.0,...,0.0,940,0,0.0,0.0,0.000,0.000,930.6,0,0.00
2,2675356,2020-01-03,3,NaN,NaN,10.0,NaN,NaN,NaN,0.0,...,639.9,0,0,0.0,0.0,0.000,614.304,0.0,0,0.00
3,2675357,2020-01-04,4,50.0,52.666667,NaN,NaN,NaN,NaN,38.0,...,0.0,0,0,0.0,2376.5,2554.495,0.000,0.0,0,0.00
4,2675358,2020-01-05,5,NaN,NaN,NaN,40.0,NaN,NaN,0.0,...,0.0,940,0,0.0,0.0,0.000,0.000,930.6,0,0.00


In [ ]:
data_posting_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   BUSINESS_DT         4 non-null      datetime64[ns]
 1   ACCOUNT_DEBIT_ID    4 non-null      int64         
 2   POSTING_AMT         4 non-null      int64         
 3   ACCOUNT_CREDIT_ID   4 non-null      int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 256.0 bytes


In [ ]:
data_clients_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   row_id             10 non-null     int64         
 1   registration_date  10 non-null     datetime64[ns]
 2   fst_name           10 non-null     object        
 3   last_name          10 non-null     object        
 4   mid_name           10 non-null     object        
 5   sex_mf             10 non-null     object        
 6   card_no            10 non-null     int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 688.0+ bytes


In [ ]:
data_pet_chk_20_pet_chk_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   card_no         35 non-null     int64         
 1   operation_date  35 non-null     datetime64[ns]
 2   cheque_id       35 non-null     int64         
 3   vol_ai92        11 non-null     float64       
 4   vol_ai95        11 non-null     float64       
 5   vol_ai98        7 non-null      float64       
 6   vol_gaz         8 non-null      float64       
 7   vol_diesel      7 non-null      float64       
 8   vol_other       6 non-null      float64       
 9   wt_ai92         35 non-null     float64       
 10  wt_ai95         35 non-null     float64       
 11  wt_ai98         35 non-null     float64       
 12  wt_gaz          35 non-null     float64       
 13  wt_diesel       35 non-null     float64       
 14  wt_other        35 non-null     float64       
 15  a_ai92  

In [ ]:
data_posting_table.columns


Index(['BUSINESS_DT ', 'ACCOUNT_DEBIT_ID ', 'POSTING_AMT ',
       'ACCOUNT_CREDIT_ID '],
      dtype='object')

In [ ]:
data_clients_info.columns

Index(['row_id', 'registration_date', 'fst_name', 'last_name', 'mid_name',
       'sex_mf', 'card_no'],
      dtype='object')

In [ ]:
data_pet_chk_20_pet_chk_21.columns

Index(['card_no', 'operation_date', 'cheque_id', 'vol_ai92', 'vol_ai95',
       'vol_ai98', 'vol_gaz', 'vol_diesel', 'vol_other', 'wt_ai92', 'wt_ai95',
       'wt_ai98', 'wt_gaz', 'wt_diesel', 'wt_other', 'a_ai92', 'a_ai95',
       'a_ai98', 'a_gaz', 'a_diesel', 'a_other', 'ta_ai92', 'ta_ai95',
       'ta_ai98', 'ta_gaz', 'ta_diesel', 'ta_other'],
      dtype='object')

In [ ]:
data_posting_table.rename(columns = {'BUSINESS_DT ':'BUSINESS_DT',
                     'ACCOUNT_DEBIT_ID ':'ACCOUNT_DEBIT_ID',
                     'POSTING_AMT ':'POSTING_AMT',
                     'ACCOUNT_CREDIT_ID ':'ACCOUNT_CREDIT_ID'}, inplace = True )

In [ ]:
data_posting_table.columns

Index(['BUSINESS_DT', 'ACCOUNT_DEBIT_ID', 'POSTING_AMT', 'ACCOUNT_CREDIT_ID'], dtype='object')

In [ ]:
posting_table = data_posting_table.to_csv('posting_table.csv')
clients_info = data_clients_info.to_csv('clients_info.csv')
pet_chk_20_pet_chk_21 = data_pet_chk_20_pet_chk_21.to_csv('pet_chk_20_pet_chk_21.csv')

## SQL запросы

In [ ]:
# загрузка данных
posting_table = spark.read.load('/content/drive/My Drive/Colab Notebooks/КАРЬЕРНЫЙ ТРЕК/11.01 Data Scientist (ЛУКОЙЛ)/posting_table.csv',
                       format='csv', header='true', inferSchema='true').createOrReplaceTempView('posting_table')

In [ ]:
spark.sql(
    "SELECT *\
    FROM posting_table"
    ).show()

+---+-------------------+----------------+-----------+-----------------+
|_c0|        BUSINESS_DT|ACCOUNT_DEBIT_ID|POSTING_AMT|ACCOUNT_CREDIT_ID|
+---+-------------------+----------------+-----------+-----------------+
|  0|2020-01-01 00:00:00|               1|       1000|                2|
|  1|2020-01-05 00:00:00|               2|        500|                1|
|  2|2020-02-04 00:00:00|               2|        500|                1|
|  3|2020-03-04 00:00:00|               1|       1000|                2|
+---+-------------------+----------------+-----------+-----------------+



### 1. Написать SQL-запрос, который на основании таблицы проводок рассчитает остатки по счетам на текущую дату. Начальные остатки по всем счетам полагаем равными нулю. Таблица должна состоять из следующих столбцов:

*	Account_ID – идентификатор счета
* Current_Date – дата, на которую посчитан остаток
*	Account_Balance – остаток на счете, может быть отрицательным.

За идентификатор счета берем значения счета дебита, далее мы определяем командой текущую дату и после этого подсчитываем остаток на счете


**SQL-запрос, который на основании таблицы проводок рассчитает остатки по счетам на текущую дату:**

In [ ]:
%%time
spark.sql(
    "WITH\
    s1 AS (SELECT CAST(BUSINESS_DT AS date) AS DT, ACCOUNT_CREDIT_ID AS Account_ID, POSTING_AMT AS Money\
    FROM posting_table\
    UNION ALL\
    SELECT CAST(BUSINESS_DT AS date) AS DT, ACCOUNT_DEBIT_ID AS AccountID, POSTING_AMT *  (-1) AS Money\
    FROM posting_table)\
    \
    SELECT Account_ID, DT AS Current_Date,\
    SUM(Money) OVER(PARTITION BY Account_ID ORDER BY DT) AS Account_Balance\
    FROM s1\
    ORDER BY DT"
    ).show()

+----------+------------+---------------+
|Account_ID|Current_Date|Account_Balance|
+----------+------------+---------------+
|         1|  2020-01-01|          -1000|
|         2|  2020-01-01|           1000|
|         1|  2020-01-05|           -500|
|         2|  2020-01-05|            500|
|         1|  2020-02-04|              0|
|         2|  2020-02-04|              0|
|         1|  2020-03-04|          -1000|
|         2|  2020-03-04|           1000|
+----------+------------+---------------+

CPU times: user 12.6 ms, sys: 1.58 ms, total: 14.2 ms
Wall time: 1.04 s


### 2. Написать SQL-запрос, который выводит таблицу остатков за весь период, за который нам известны проводки (например, с января по март). Начальные остатки по всем счетам полагаем равными нулю. Таблица должна состоять из следующих столбцов:

* Account_ID – идентификатор счета
* Business_From_DT – дата, начиная с которой (включительно) сформировался указанный остаток на счете.
* Business_To_DT – дата, по которую (включительно) на счете находится указанный остаток. Если остаток действует по текущий момент (дата следующего изменения остатка не известна), то поле следует заполнить датой 9999-12-31.
* Account_Balance – остаток на счете Account_ID в период с Business_From_DT по Business_To_DT включительно.

Например, на счете 2 был остаток в 1000 рублей, начиная с 01.01.2020 по 04.01.2020 включительно (5-го числа остаток уже поменялся).


**SQL-запрос, который выводит таблицу остатков за весь период, за который нам известны проводки (например, с января по март):**

In [ ]:
%%time
spark.sql(
    "WITH\
    s2 AS (WITH\
    s1 AS (SELECT CAST(BUSINESS_DT AS date) AS DT, ACCOUNT_CREDIT_ID AS Account_ID, POSTING_AMT AS Money\
    FROM posting_table\
    UNION ALL\
    SELECT CAST(BUSINESS_DT AS date) AS DT, ACCOUNT_DEBIT_ID AS AccountID, POSTING_AMT *  (-1) AS Money\
    FROM posting_table)\
    \
    SELECT Account_ID, DT AS Current_Date,\
    SUM(Money) OVER(PARTITION BY Account_ID ORDER BY DT) AS Account_Balance\
    FROM s1\
    ORDER BY DT)\
    \
    SELECT Account_ID,\
    Current_Date AS Business_From_DT,\
    LEAD(Current_Date, 1, '10000-01-01') OVER(PARTITION BY Account_ID ORDER BY Current_Date) - 1 AS Business_To_DT ,\
    Account_Balance\
    FROM s2\
    ORDER BY Business_From_DT"
    ).show()

+----------+----------------+--------------+---------------+
|Account_ID|Business_From_DT|Business_To_DT|Account_Balance|
+----------+----------------+--------------+---------------+
|         1|      2020-01-01|    2020-01-04|          -1000|
|         2|      2020-01-01|    2020-01-04|           1000|
|         1|      2020-01-05|    2020-02-03|           -500|
|         2|      2020-01-05|    2020-02-03|            500|
|         1|      2020-02-04|    2020-03-03|              0|
|         2|      2020-02-04|    2020-03-03|              0|
|         1|      2020-03-04|    9999-12-31|          -1000|
|         2|      2020-03-04|    9999-12-31|           1000|
+----------+----------------+--------------+---------------+

CPU times: user 11.1 ms, sys: 6.09 ms, total: 17.2 ms
Wall time: 1.81 s


### 3. Написать SQL запрос, который выведет остатки на каждый день за период с января по март. Т.е. по каждому счету таблица должна содержать около 90 записей. Начальные остатки по всем счетам полагаем равными нулю. Столбцы таблицы:

* Account_ID – идентификатор счета
* Business_Date – дата, на которую посчитан остаток
* Account_Balance – остаток на счете, может быть отрицательным.

Например, по счету 2 должны быть такие записи:

* 2, 2020-01-01, 1000
* 2, 2020-01-02, 1000
* 2, 2020-01-03, 1000
* 2, 2020-01-04, 1000
* 2, 2020-01-05, 500

И т.д.


In [ ]:
# кадр дат с '2020-01-01' по '2020-03-04'
tab_a = spark.sql("SELECT sequence(to_date('2020-01-01'), to_date('2020-03-04'), interval 1 day) as date")\
.withColumn("date", explode(col("date")))

In [ ]:
# вывод кадра дат
tab_a.show()

+----------+
|      date|
+----------+
|2020-01-01|
|2020-01-02|
|2020-01-03|
|2020-01-04|
|2020-01-05|
|2020-01-06|
|2020-01-07|
|2020-01-08|
|2020-01-09|
|2020-01-10|
|2020-01-11|
|2020-01-12|
|2020-01-13|
|2020-01-14|
|2020-01-15|
|2020-01-16|
|2020-01-17|
|2020-01-18|
|2020-01-19|
|2020-01-20|
+----------+
only showing top 20 rows



In [ ]:
# загрузка фрейма дат в spark
DT_tab = spark.read.load('/content/drive/My Drive/Colab Notebooks/КАРЬЕРНЫЙ ТРЕК/11.01 Data Scientist (ЛУКОЙЛ)/DT_tab.csv',
                       format='csv', header='true', inferSchema='true').createOrReplaceTempView('DT_tab')

**SQL запрос, который выведет остатки на каждый день за период с января по март:**

In [ ]:
%%time
spark.sql(
    "WITH\
    s4 AS (WITH\
    s2 AS (WITH\
    s1 AS (SELECT CAST(BUSINESS_DT AS date) AS DT, ACCOUNT_CREDIT_ID AS Account_ID, POSTING_AMT AS Money\
    FROM posting_table\
    UNION ALL\
    SELECT CAST(BUSINESS_DT AS date) AS DT, ACCOUNT_DEBIT_ID AS AccountID, POSTING_AMT *  (-1) AS Money\
    FROM posting_table)\
    \
    SELECT Account_ID, DT AS Current_Date,\
    SUM(Money) OVER(ORDER BY DT) AS Account_Balance\
    FROM s1\
    WHERE Account_ID = 1\
    ORDER BY Current_Date),\
    \
    s3 AS (SELECT *\
    FROM DT_tab)\
    \
    SELECT CAST(s3.date AS date) AS Current_Date,\
    s2.Account_ID, s2.Account_Balance\
    FROM s3\
    FULL JOIN s2 ON s2.Current_Date=s3.date)\
    \
    SELECT Current_Date,\
    LAG(Account_ID, 1, 1) OVER(PARTITION BY Current_Date ORDER BY Current_Date) AS Account_ID,\
    LAST_VALUE(Account_Balance) IGNORE NULLS OVER (ORDER BY Current_Date) AS Account_Balance_now\
    FROM s4"
    ).show()

    # ПРИМЕЧАНИЕ:
    # Для просмотра выгрузки по кошельку 2 достаточно поменять в 13, 24 строках
    # 'WHERE Account_ID = 1' на 'WHERE Account_ID = 2' и LAG(Account_ID, 1, 1) на LAG(Account_ID, 1, 2)

+------------+----------+-------------------+
|Current_Date|Account_ID|Account_Balance_now|
+------------+----------+-------------------+
|  2020-01-01|         1|              -1000|
|  2020-01-02|         1|              -1000|
|  2020-01-03|         1|              -1000|
|  2020-01-04|         1|              -1000|
|  2020-01-05|         1|               -500|
|  2020-01-06|         1|               -500|
|  2020-01-07|         1|               -500|
|  2020-01-08|         1|               -500|
|  2020-01-09|         1|               -500|
|  2020-01-10|         1|               -500|
|  2020-01-11|         1|               -500|
|  2020-01-12|         1|               -500|
|  2020-01-13|         1|               -500|
|  2020-01-14|         1|               -500|
|  2020-01-15|         1|               -500|
|  2020-01-16|         1|               -500|
|  2020-01-17|         1|               -500|
|  2020-01-18|         1|               -500|
|  2020-01-19|         1|         

### 4. Структура данных в таблицах схемы public:

*pet_chk_20, pet_chk_21 - данные по реализации: чеки агрегированные - только карты лояльности*

* card_no	- Номер карты
* org_ksss - КССС АЗС
* npo	- НПО
* org_name	- АЗС
* dt	- Дата
* operation_date	- Дата/Время
* cheque_id	- ID чека
* cnt_pos	- Количество позиций в чеке
* cnt_pos_ntu	- Количество позиций в чеке НТУ
* cnt_pos_fuel	- Количество позиций в чеке Топлива
* cnt_pos_ai92	- Количество позиций в чеке Топлива АИ-92
* cnt_pos_ai95	- Количество позиций в чеке Топлива АИ-95
* cnt_pos_ai98	- Количество позиций в чеке Топлива АИ-98
* cnt_pos_gaz	- Количество позиций в чеке Топлива СУГ
* cnt_pos_diesel	- Количество позиций в чеке Топлива Дизель
* cnt_pos_other	- Количество позиций в чеке Топлива Другое
* cnt_unq	- Количество уникальных товаров
* cnt_unq_ntu	- Количество уникальных товаров НТУ
* cnt_unq_fuel	- Количество уникальных товаров Топлива
* qnt	- Количество товаров НТУ
* ltr_vol	- Количество литров Топлива
* vol_ai92	- Количество литров Топлива АИ-92
* vol_ai95	- Количество литров Топлива АИ-95
* vol_ai98	- Количество литров Топлива АИ-98
* vol_gaz	- Количество литров Топлива СУГ
* vol_diesel	- Количество литров Топлива Дизель
* vol_other	- Количество литров Топлива Другое
* weight	- Вес
* wt_ntu	- Вес НТУ
* wt_ai92	- Вес Топлива АИ-92
* wt_ai95	- Вес Топлива АИ-95
* wt_ai98	- Вес Топлива АИ-98
* wt_gaz	- Вес Топлива СУГ
* wt_diesel	- Вес Топлива Дизель
* wt_other	- Вес Топлива Другое
* amount	- Сумма без скидки
* a_ntu	- Сумма без скидки НТУ
* a_fuel	- Сумма без скидки Топлива
* a_ai92	- Сумма без скидки Топлива АИ-92
* a_ai95	- Сумма без скидки Топлива АИ-95
* a_ai98	- Сумма без скидки Топлива АИ-98
* a_gaz	- Сумма без скидки Топлива СУГ
* a_diesel	- Сумма без скидки Топлива Дизель
* a_other	- Сумма без скидки Топлива Другое
* target_amount	- Сумма с скидкой
* ta_ntu	- Сумма с скидкой НТУ
* ta_fuel	- Сумма с скидкой Топлива
* ta_ai92	- Сумма с скидкой Топлива АИ-92
* ta_ai95	- Сумма с скидкой Топлива АИ-95
* ta_ai98	- Сумма с скидкой Топлива АИ-98
* ta_gaz	- Сумма с скидкой Топлива СУГ
* ta_diesel	- Сумма с скидкой Топлива Дизель
* ta_other	- Сумма с скидкой Топлива Другое

*clients_info – персональные данные о клиенте программы лояльности*

* row_id	- id записи
* registration_date	- дата регистрации
* fst_name	- фамилия
* last_name	- имя
* mid_name	- отчество
* sex_mf	- пол
* birth_dt	- дата рождения
* card_no	- номер кл
* phone_num	- номер мобильного телефона
* sms_agreement	- согласие на смс
* email_addr	- адрес электронной почты
* email_agreement	- согласие на email рассылку
* push_agreement	- согласие на пуш уведомления
* LAST_UPD	- дата последнего обновления

pet_chk_20.card_no = pet_chk_21.card_no = clients_info.card_no

*Задания:*

1. Распределение реализации по видам топлива по Картам Лояльности в 2020 и 2021 гг.
2. Определить средний чек в литрах по клиентам программы лояльности в разрезе полового признака в 2021г.
3. Выгрузить распределение количества чеков по клиентам программы лояльности шагом в 50 литров в январе 2021 года.


In [ ]:
# загрузка clients_info
clients_info = spark.read.load('/content/drive/My Drive/Colab Notebooks/КАРЬЕРНЫЙ ТРЕК/11.01 Data Scientist (ЛУКОЙЛ)/clients_info.csv',
                       format='csv', header='true', inferSchema='true').createOrReplaceTempView('clients_info')

In [ ]:
# загрузка pet_chk_20_pet_chk_21
pet_chk_20_pet_chk_21 = spark.read.load('/content/drive/My Drive/Colab Notebooks/КАРЬЕРНЫЙ ТРЕК/11.01 Data Scientist (ЛУКОЙЛ)/pet_chk_20_pet_chk_21.csv',
                       format='csv', header='true', inferSchema='true').createOrReplaceTempView('pet_chk_20_pet_chk_21')

In [ ]:
# вывод clients_info
spark.sql(
    "SELECT *\
    FROM clients_info"
    ).show()

+---+------+-------------------+---------+---------+----------+------+-------+
|_c0|row_id|  registration_date| fst_name|last_name|  mid_name|sex_mf|card_no|
+---+------+-------------------+---------+---------+----------+------+-------+
|  0|     1|2019-04-01 00:00:00|   Иванов|     Петр| Сергеевич|  male|2675354|
|  1|     2|2019-05-02 00:00:00|   Петров|   Сергей|  Иванович|  male|2675355|
|  2|     3|2018-06-03 00:00:00|  Семенов|    Семен| Семенович|  male|2675356|
|  3|     4|2017-07-05 00:00:00|  Сергеев|     Иван|  Петрович|  male|2675357|
|  4|     5|2016-08-06 00:00:00| Синицина|    Ольга|  Ивановна|female|2675358|
|  5|     6|2019-04-01 00:00:00| Клещеева| Наталья |  Петровна|female|2675359|
|  6|     7|2015-05-02 00:00:00|Дявяткина| Светлана| Семеновна|female|2675360|
|  7|     8|2018-01-07 00:00:00| Харламов|Александр|  Иванович|  male|2675361|
|  8|     9|2015-09-14 00:00:00|  Кошкина|   Дарья |Николаевна|female|2675362|
|  9|    10|2013-05-21 00:00:00| Дергунов|   Артем |

In [ ]:
# вывод pet_chk_20_pet_chk_21
spark.sql(
    "SELECT *\
    FROM pet_chk_20_pet_chk_21\
    LIMIT(3)"
    ).show()

+---+-------+-------------------+---------+--------+--------+--------+-------+----------+---------+-------+-------+-----------------+------------------+---------+--------+------+------+------+-----+--------+-------+-------+-------+-------+------+---------+--------+
|_c0|card_no|     operation_date|cheque_id|vol_ai92|vol_ai95|vol_ai98|vol_gaz|vol_diesel|vol_other|wt_ai92|wt_ai95|          wt_ai98|            wt_gaz|wt_diesel|wt_other|a_ai92|a_ai95|a_ai98|a_gaz|a_diesel|a_other|ta_ai92|ta_ai95|ta_ai98|ta_gaz|ta_diesel|ta_other|
+---+-------+-------------------+---------+--------+--------+--------+-------+----------+---------+-------+-------+-----------------+------------------+---------+--------+------+------+------+-----+--------+-------+-------+-------+-------+------+---------+--------+
|  0|2675354|2020-01-01 00:00:00|        1|    null|    null|    null|   null|      null|     10.0|    0.0|    0.0|              0.0|               0.0|      0.0|     7.4|   0.0|   0.0|   0.0|    0|    

**1)	Распределение реализации по видам топлива по Картам Лояльности в 2020 и 2021 гг.**

In [ ]:
spark.sql(
    "SELECT card_no, EXTRACT(year FROM CAST(operation_date AS date)) AS on_years,\
    ROUND(SUM(ta_ai92), 2) AS realiz_in_rub_ai92,\
    ROUND(SUM(ta_ai95), 2) AS realiz_in_rub_ai95,\
    ROUND(SUM(ta_ai98), 2) AS realiz_in_rub_ai98,\
    ROUND(SUM(ta_gaz), 2) AS realiz_in_rub_gaz,\
    ROUND(SUM(ta_diesel), 2) AS realiz_in_rub_diesel,\
    ROUND(SUM(ta_other), 2) AS realiz_in_rub_other\
    FROM pet_chk_20_pet_chk_21\
    GROUP BY on_years, card_no\
    ORDER BY on_years"
    ).show()

+-------+--------+------------------+------------------+------------------+-----------------+--------------------+-------------------+
|card_no|on_years|realiz_in_rub_ai92|realiz_in_rub_ai95|realiz_in_rub_ai98|realiz_in_rub_gaz|realiz_in_rub_diesel|realiz_in_rub_other|
+-------+--------+------------------+------------------+------------------+-----------------+--------------------+-------------------+
|2675363|    2020|               0.0|               0.0|               0.0|              0.0|                   0|             738.92|
|2675361|    2020|            1901.2|            2043.6|               0.0|              0.0|                   0|                0.0|
|2675360|    2020|               0.0|               0.0|               0.0|            930.6|                   0|                0.0|
|2675355|    2020|               0.0|               0.0|               0.0|            930.6|                   0|                0.0|
|2675362|    2020|               0.0|               0.0

**2)	Определить средний чек в литрах по клиентам программы лояльности в разрезе полового признака в 2021 г.**

In [ ]:
spark.sql(
    "WITH\
    s1 AS (SELECT card_no,\
    EXTRACT(year FROM CAST(operation_date AS date)) AS on_years,\
    ROUND(AVG(vol_ai92), 2) AS avg_check_in_l_ai92,\
    ROUND(AVG(vol_ai95), 2) AS avg_check_in_l_ai95,\
    ROUND(AVG(vol_ai98), 2) AS avg_check_in_l_ai98,\
    ROUND(AVG(vol_gaz), 2) AS avg_check_in_l_gaz,\
    ROUND(AVG(vol_diesel), 2) AS avg_check_in_l_diesel,\
    ROUND(AVG(vol_other), 2) AS avg_check_in_l_other\
    FROM pet_chk_20_pet_chk_21\
    GROUP BY card_no, on_years),\
    \
    s2 AS (SELECT sex_mf, card_no\
    FROM clients_info)\
    \
    SELECT s1.card_no,\
    s2.sex_mf,\
    s1.on_years,\
    s1.avg_check_in_l_ai92,\
    s1.avg_check_in_l_ai95,\
    s1.avg_check_in_l_ai98,\
    s1.avg_check_in_l_gaz,\
    s1.avg_check_in_l_diesel,\
    s1.avg_check_in_l_other\
    FROM s1\
    JOIN s2 ON s1.card_no=s2.card_no\
    WHERE on_years = '2021'"
    ).show()

+-------+------+--------+-------------------+-------------------+-------------------+------------------+---------------------+--------------------+
|card_no|sex_mf|on_years|avg_check_in_l_ai92|avg_check_in_l_ai95|avg_check_in_l_ai98|avg_check_in_l_gaz|avg_check_in_l_diesel|avg_check_in_l_other|
+-------+------+--------+-------------------+-------------------+-------------------+------------------+---------------------+--------------------+
|2675363|  male|    2021|               45.0|              45.67|               50.0|              30.0|                 null|                null|
|2675362|female|    2021|               50.0|               null|               null|              null|                 null|                null|
|2675358|female|    2021|              29.89|              54.11|              51.67|              null|                 35.0|                30.0|
|2675359|female|    2021|               null|               null|               null|              null|        

**3)	Выгрузить распределение количества чеков по клиентам программы лояльности шагом в 50 литров в январе 2021 года.**

In [ ]:
spark.sql(
    "SELECT card_no,\
    ROUND(COUNT(vol_ai92)/SUM(vol_ai92)/50, 4) AS ct_check_50_l_ai92,\
    ROUND(COUNT(vol_ai95)/SUM(vol_ai95)/50, 4) AS ct_check_50_l_ai95,\
    ROUND(COUNT(vol_ai98)/SUM(vol_ai98)/50, 4) AS ct_check_50_l_ai98,\
    ROUND(COUNT(vol_gaz)/SUM(vol_gaz)/50, 4) AS ct_check_50_l_gaz,\
    ROUND(COUNT(vol_diesel)/SUM(vol_diesel)/50, 4) AS ct_check_50_l_diesel,\
    ROUND(COUNT(vol_other)/SUM(vol_other)/50, 4) AS ct_check_50_l_other\
    FROM pet_chk_20_pet_chk_21\
    WHERE (CAST(operation_date AS date) BETWEEN '2021-01-01' AND '2021-01-31')\
    GROUP BY card_no"
    ).show()

+-------+------------------+------------------+------------------+-----------------+--------------------+-------------------+
|card_no|ct_check_50_l_ai92|ct_check_50_l_ai95|ct_check_50_l_ai98|ct_check_50_l_gaz|ct_check_50_l_diesel|ct_check_50_l_other|
+-------+------------------+------------------+------------------+-----------------+--------------------+-------------------+
|2675363|            4.0E-4|            4.0E-4|            4.0E-4|           7.0E-4|                null|               null|
|2675361|             0.001|              null|              null|             null|                null|               null|
|2675368|            5.0E-4|            4.0E-4|              null|             null|                null|               null|
|2675360|              null|              null|              null|           7.0E-4|                null|               null|
|2675362|            4.0E-4|              null|              null|             null|                null|             